In [1]:
library(dbplyr)
library(dplyr)
library(odbc)
library(plutoR)
options("scipen"=999)
source("config.R")


Attaching package: ‘dplyr’

The following objects are masked from ‘package:dbplyr’:

    ident, sql

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



### get all total-return NSE indices for the latest time_stamp

In [2]:
indices<-Indices()

In [3]:
maxDtNse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtNse && NAME %like% '% TR%') %>%
     select(NAME) %>%
     print(n = Inf)

Warning message:
“Missing values are always removed in SQL.
Use `MAX(x, na.rm = TRUE)` to silence this warning
This warning is displayed only once per session.”

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME                                          
   <chr>                                         
 1 NIFTY 100 TR                                  
 2 NIFTY 200 TR                                  
 3 NIFTY 50 TR                                   
 4 NIFTY 500 TR                                  
 5 NIFTY ADITYA BIRLA GROUP TR                   
 6 NIFTY ALPHA 50 TR                             
 7 NIFTY ALPHA LOW-VOLATILITY 30 TR              
 8 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR      
 9 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR
10 NIFTY AUTO TR                                 
11 NIFTY BANK TR                                 
12 NIFTY COMMODITIES TR                          
13 NIFTY CPSE TR                                 
14 NIFTY DIVIDEND OPPORTUNITIES 50 TR            
15 NIFTY ENERGY TR                               
16 NIFTY FINANCIAL SERVICES TR      

### get all BSE indices for the latest time_stamp

In [4]:
maxDtBse <- (indices$BseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$BseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n = Inf)

# Source:   lazy query [?? x 1]
# Database: Microsoft SQL Server 13.00.4224[ro1@NORWAY/StockViz]
# Groups:   NAME
   NAME    
   <chr>   
 1 BSE100  
 2 BSE200  
 3 BSE500  
 4 BSEMID  
 5 BSESML  
 6 CARBON  
 7 CPSE    
 8 DOL100  
 9 DOLL30  
10 DOLLEX  
11 GREENX  
12 INFRA   
13 IPO     
14 SENSEX  
15 SI0200  
16 SI0400  
17 SI0600  
18 SI0800  
19 SI1000  
20 SI1200  
21 SI1400  
22 SI1900  
23 SIBANK  
24 SIBPSU  
25 SIBTEC  
26 SIPOWE  
27 SIREAL  
28 SMEIPO  
29 SPB15MIP
30 SPB1LTIP
31 SPB25SIP
32 SPB25XIP
33 SPB2MXIP
34 SPB40YIP
35 SPBDISBT
36 SPBIMIP 
37 SPBMDSIP
38 SPBN5TIP
39 SPBS1E  
40 SPBS5TIP
41 SPBSAIP 
42 SPBSB2IP
43 SPBSBMIP
44 SPBSCDIP
45 SPBSDFIP
46 SPBSDSIP
47 SPBSE5S 
48 SPBSENIP
49 SPBSEVIP
50 SPBSFIIP
51 SPBSIDIP
52 SPBSLIP 
53 SPBSLMIP
54 SPBSLVIP
55 SPBSMOIP
56 SPBSMSIP
57 SPBSN5IP
58 SPBSPBIP
59 SPBSQIP 
60 SPBSS5IP
61 SPBSSSIP
62 SPBSTLIP
63 SPBSUTIP
64 SPICBLOT
